In [2]:
# import necessary packages
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import geopandas as gpd
from descartes import PolygonPatch
from tqdm import tqdm

In [3]:
df = pd.read_csv("../../Raw_Data/full-run-aggregation-20-table.zip",skiprows=6)

In [5]:
#remove all the columns that have the same value in all rows.
for c in df.columns:
    if len(df[c].unique()) < 2:
        df = df.drop(columns=c)

In [27]:
#remove all rows with no data, these are usually the rows at the start of a run.
l = []
for r in df.index:
    if len(df.iloc[r,8:].astype(str).unique())<2:
        l.append(r)

df = df.drop(index=l)

In [8]:
#I found this code to make dict from long list.
def listToDict(lst):
    op = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return op

In [28]:
df.shape

(81920, 30)

In [29]:
#This will take two eons of time, please don't run this code unless you really need to.
steps = []
n_steps = 69696969
for index, row in tqdm(df.iloc[:n_steps,:].iterrows(), total=n_steps):
    l = []
    for column in df.columns[:10]:
        x = row[column]
        new_df = pd.DataFrame(index=np.arange(0,114), columns=[column])
        new_df[column] = x
        l.append(new_df)
    for column in df.columns[10:]:
        x = row[column].replace("[","").replace("]","").split(" ")
        d = listToDict(x)
        new_df = pd.DataFrame(index=np.arange(0, len(d)), columns=['Neighborhood number', column])
        new_df['Neighborhood number'] = [*d]
        new_df[column] = list(d.values())
        l.append(new_df)
    steps.append(pd.concat(l, axis=1))
full_df = pd.concat(steps).loc[:,~pd.concat(steps).columns.duplicated()].convert_dtypes()

  0%|                                                                     | 81920/69696969 [53:36<759:20:22, 25.47it/s]


In [30]:
full_df = full_df.astype(float)

In [32]:
full_df.to_csv("../../Raw_Data/full_df.csv")